In [1]:
# Import dependencies
from bs4 import BeautifulSoup as soup
from splinter import Browser
import pandas as pd
import time

In [2]:
# Definte years
string_2021 = ".y2021.html"
string_2020 = ".y2020.html"

In [3]:
url_list = ['https://www.pgatour.com/content/pgatour/stats/stat.02567',
'https://www.pgatour.com/content/pgatour/stats/stat.02674',
'https://www.pgatour.com/content/pgatour/stats/stat.101',
'https://www.pgatour.com/content/pgatour/stats/stat.02341',
'https://www.pgatour.com/content/pgatour/stats/stat.496']

In [4]:
List_2021 = [s + string_2021 for s in url_list]
List_2020 = [s + string_2020 for s in url_list]

In [5]:
# Start out dataframes as empty lists (IDs listed in golfstats_FINAL_Excel file)
# NOTE: This did not work when I made a list of lists using df = [[]] * 23 because you need to define
## the lists in the list for the scrape function to work
## just showing 2021 and 2020 in this notebook as an example

df1 = []
df2 = []
df3 = []
df4 = []
df5 = []
df6 = []
df7 = []
df8 = []
df9 = []
df10 = []

dataframe_2021 = [df1,df2,df3,df4,df5]
dataframe_2020 = [df6,df7,df8,df9,df10]

In [6]:
# Define scrape function using dynamic dataframe variable names
## This method makes the above step necessary

def scrape(List_year, df):
    
    # Enumerate url list and loop through it
    for index, url in enumerate(List_year):
        
        # Print start
        start = time.time()

        print(f"Start loop {index+1}")

         # Set the executable path and initialize the chrome browser in splinter
        executable_path = {'executable_path': 'chromedriver'}
        browser = Browser('chrome', **executable_path)
    
        # Pause 3 seconds so as not to overwhelm server
        time.sleep(3)
    
        # Visit URL, parse, find table
        browser.visit(url)
        browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
        html = browser.html
        golf_soup = soup(html, 'html.parser')
        table = golf_soup.find('table', class_='table-styled')
    
        # Find rows and headings
        table_rows = table.find('tbody').find_all('tr')
        table_headings = table.find('thead').find_all('th')
    
        # Strip column name text and append to dfs
        column_names = [cm.text.strip() for cm in table_headings]
        df[index].append(column_names)
    
        # Strip row text and append to dfs
        for tr in table_rows:
            rows = [tr.text.strip() for tr in tr.find_all('td')]
            df[index].append(rows)
    
        # Convert list to pandas dataframe
        df[index]= pd.DataFrame(df[index])
    
        # Promote headers
        header = df[index].iloc[0]
        df[index] = df[index][1:]
        df[index].columns = header
    
        # Set Index equal to PLAYER NAME
        df[index] = df[index].set_index('PLAYER NAME')
    
        # Quit browser
        browser.quit()
    
        # Print progress
        print(f"End loop {index+1} | Elapsed loop time: {time.time() - start}")

In [7]:
# Run scrape 2021
scrape(List_2021, dataframe_2021)

Start loop 1
End loop 1 | Elapsed loop time: 14.76287579536438
Start loop 2
End loop 2 | Elapsed loop time: 16.69131374359131
Start loop 3
End loop 3 | Elapsed loop time: 13.922194004058838
Start loop 4
End loop 4 | Elapsed loop time: 16.66594433784485
Start loop 5
End loop 5 | Elapsed loop time: 15.548963069915771


In [8]:
# Run scrape 2020
scrape(List_2020, dataframe_2020)

Start loop 1
End loop 1 | Elapsed loop time: 15.853954553604126
Start loop 2
End loop 2 | Elapsed loop time: 20.165733814239502
Start loop 3
End loop 3 | Elapsed loop time: 17.83914566040039
Start loop 4
End loop 4 | Elapsed loop time: 15.73022985458374
Start loop 5
End loop 5 | Elapsed loop time: 20.11806583404541


In [9]:
# Rename all columns to make them unique (took this code from our Excel formulas)
# Renaming each dataframe using this method is necessary because of how I needed
## to do it above to make the scrape function work the way I wanted
df1=dataframe_2021[0][['AVERAGE']].rename(columns={'AVERAGE': 'SG_off_tee_2021_AVERAGE'})
df2=dataframe_2021[1][['AVERAGE']].rename(columns={'AVERAGE': 'SG_tee_green_2021_AVERAGE'})
df3=dataframe_2021[2][['AVG.']].rename(columns={'AVG.': 'driving_dist_2021_AVG.'})
df4=dataframe_2021[3][['AVG (%)']].rename(columns={'AVG (%)': 'pct_ydg_tee_2021_AVG (%)'})
df5=dataframe_2021[4][['%']].rename(columns={'%': 'driving_320+_2021_%'})

In [10]:
# Rename 2020
df6=dataframe_2020[0][['AVERAGE']].rename(columns={'AVERAGE': 'SG_off_tee_2020_AVERAGE'})
df7=dataframe_2020[1][['AVERAGE']].rename(columns={'AVERAGE': 'SG_tee_green_2020_AVERAGE'})
df8=dataframe_2020[2][['AVG.']].rename(columns={'AVG.': 'driving_dist_2020_AVG.'})
df9=dataframe_2020[3][['AVG (%)']].rename(columns={'AVG (%)': 'pct_ydg_tee_2020_AVG (%)'})
df10=dataframe_2020[4][['%']].rename(columns={'%': 'driving_320+_2020_%'})

In [11]:
# Join all on index already set to create final dataset
final_df = df1.join(df2).join(df3).join(df4).join(df5).join(df6).join(df7).join(df8).join(df9).join(df10)

In [12]:
# Reformat names with symbols
new_name_format = {'Sebastián Muñoz':'Sebastian Munoz',\
             'Fabián Gómez':'Fabian Gomez'}
           
final_df = final_df.rename(index=new_name_format)

In [13]:
# Drop all blank rows (some players have only played one year and we're choosing to exclude them)
full_final_df = final_df.dropna()

In [14]:
# Found code on Stack Overflow and ammended it to convert measurement columns to numbers without symbols
# Didn't include these columns in the rewrite, but if you had the columns, this is how you'd do it
def parse_measurements(i):
    measurement_ = i.split("' ")
    ft_ = float(measurement_[0])
    in_ = float(measurement_[1].replace("\"",""))
    return (12*ft_) + in_

# # Comment out because I didn't include these columns in the rewrite
# full_final_df['fwy_prox_2021_AVG'] = full_final_df['fwy_prox_2021_AVG'].apply(lambda x:parse_measurements(x))
# full_final_df['rough_prox_2021_AVG'] = full_final_df['rough_prox_2021_AVG'].apply(lambda x:parse_measurements(x))
# full_final_df['fwy_prox_2020_AVG'] = full_final_df['fwy_prox_2020_AVG'].apply(lambda x:parse_measurements(x))
# full_final_df['rough_prox_2020_AVG'] = full_final_df['rough_prox_2020_AVG'].apply(lambda x:parse_measurements(x))

In [15]:
# Do same thing with $ column
# Didn't include these columns in the rewrite, but if you had the columns, this is how you'd do it
def parse_money_symbol(i):
    symbol_ = i.split("$")
    number = float(symbol_[1].replace(",",""))
    return number

# # Comment out because I didn't include these columns in the rewrite
# full_final_df['money_2021_MONEY'] = full_final_df['money_2021_MONEY'].apply(lambda x: parse_money_symbol(x))
# full_final_df['money_2020_MONEY'] = full_final_df['money_2020_MONEY'].apply(lambda x: parse_money_symbol(x))

In [16]:
# Convert all columns to numberic and send to csv
cleaned_final_df = full_final_df.apply(pd.to_numeric)

# cleaned_final_df.to_csv('cleaned_dataset.csv')